# OpenAI Agents SDK - 세션 관리

이 튜토리얼은 OpenAI Agents SDK에서 세션(Session)을 사용하여 대화 히스토리를 관리하는 방법을 다룬다. 세션을 통해 여러 번의 에이전트 실행 간에도 대화 맥락을 유지할 수 있다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 세션 사용하기

이 챕터에서는 세션(Session)을 사용하여 이전 대화 내용을 기억하고 연속적인 대화를 유지하는 방법을 학습한다.

### 학습 내용

- SQLiteSession을 사용한 대화 히스토리 관리
- 세션 ID를 통한 대화 묶음 구분
- 대화 히스토리 추가 및 삭제
- 연속 대화에서 맥락 유지

### 환경 설정

In [1]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, ModelSettings, SQLiteSession

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 세션의 개념

세션(Session)은 여러 번의 에이전트 실행 간에 대화 히스토리를 자동으로 유지하는 메커니즘이다.

- **세션 ID**: 하나의 대화 묶음을 구분하는 고유한 값
- **대화 히스토리**: 이전 대화 내용을 저장하여 맥락 유지
- **SQLiteSession**: SQLite 데이터베이스를 사용한 세션 관리

In [2]:
# 기본 에이전트 생성
agent = Agent(
    name="Assistant",
    instructions="매우 간결하게 답변해 주세요. 한국어로 응답하세요.",
    model=MODEL,
    model_settings=ModelSettings(
        temperature=0.7,
        max_tokens=500
    )
)

print(f"에이전트 '{agent.name}' 생성 완료")

에이전트 'Assistant' 생성 완료


### 기본 세션 사용법

In [3]:
async def demo_basic_session():
    """
    기본 세션 사용법을 시연한다.
    세션을 통해 이전 대화 내용을 기억하고 맥락을 유지한다.
    """
    
    # 세션 객체 생성 - 세션 ID로 대화 묶음을 구분
    session = SQLiteSession("demo_conversation_001")
    
    print("=" * 60)
    print("기본 세션 사용 데모")
    print("=" * 60)
    
    # 첫 번째 질문
    print("\n📝 첫 번째 질문: 첨성대는 어느 도시에 있나요?")
    result1 = await Runner.run(
        agent,
        "첨성대는 어느 도시에 있나요?",
        session=session  # 세션 객체 전달
    )
    print(f"🤖 답변: {result1.final_output}")
    
    # 두 번째 질문 - 이전 대화를 기억해야 답변 가능
    print("\n📝 두 번째 질문: 그 도시의 관광 명소를 알려주세요.")
    result2 = await Runner.run(
        agent,
        "그 도시의 관광 명소를 알려주세요.",  # '경주'라고 직접 언급하지 않음
        session=session  # 동일한 세션 사용
    )
    print(f"🤖 답변: {result2.final_output}")
    
    print("\n" + "=" * 60)
    print("✓ 세션을 통해 이전 대화('경주')를 기억하여 답변함")
    print("=" * 60)

# 데모 실행
await demo_basic_session()

기본 세션 사용 데모

📝 첫 번째 질문: 첨성대는 어느 도시에 있나요?
🤖 답변: 첨성대는 경상북도 경주에 있습니다.

📝 두 번째 질문: 그 도시의 관광 명소를 알려주세요.
🤖 답변: 경주의 주요 관광 명소는 다음과 같습니다:

1. 불국사
2. 석굴암
3. 경주 국립박물관
4. 동궁과 월지
5. 안압지
6. 천마총
7. 황룡사터
8. 경주 양동마을

많은 역사적 유적지가 있습니다.

✓ 세션을 통해 이전 대화('경주')를 기억하여 답변함


### 세션 없이 대화할 때의 문제

세션을 사용하지 않으면 각 대화가 독립적으로 처리되어 맥락을 유지할 수 없다.

In [4]:
async def demo_without_session():
    """
    세션 없이 대화할 때의 문제를 보여준다.
    """
    
    print("=" * 60)
    print("세션 없이 대화 (맥락 유지 불가)")
    print("=" * 60)
    
    # 첫 번째 질문 - 세션 없음
    print("\n📝 첫 번째 질문: 첨성대는 어느 도시에 있나요?")
    result1 = await Runner.run(
        agent,
        "첨성대는 어느 도시에 있나요?"
        # session 파라미터 없음
    )
    print(f"🤖 답변: {result1.final_output}")
    
    # 두 번째 질문 - 세션 없음
    print("\n📝 두 번째 질문: 그 도시의 관광 명소를 알려주세요.")
    result2 = await Runner.run(
        agent,
        "그 도시의 관광 명소를 알려주세요."
        # session 파라미터 없음
    )
    print(f"🤖 답변: {result2.final_output}")
    
    print("\n" + "=" * 60)
    print("✗ 세션 없이는 '그 도시'가 무엇인지 알 수 없음")
    print("=" * 60)

# 데모 실행
await demo_without_session()

세션 없이 대화 (맥락 유지 불가)

📝 첫 번째 질문: 첨성대는 어느 도시에 있나요?
🤖 답변: 첨성대는 경주에 있습니다.

📝 두 번째 질문: 그 도시의 관광 명소를 알려주세요.
🤖 답변: 어떤 도시를 말씀하시는지 알려주시면 관광 명소를 알려드릴게요.

✗ 세션 없이는 '그 도시'가 무엇인지 알 수 없음


### 대화 히스토리 추가하기

Session 객체의 `add_items()` 메서드를 사용하여 가상의 대화 내용을 직접 추가할 수 있다.

In [5]:
async def demo_add_history():
    """
    세션에 대화 히스토리를 직접 추가하는 방법을 시연한다.
    """
    
    # 새로운 세션 생성
    session = SQLiteSession("demo_add_history_001")
    
    print("=" * 60)
    print("대화 히스토리 추가 데모")
    print("=" * 60)
    
    # 가상의 대화 히스토리 생성
    new_items = [
        {"role": "user", "content": "안녕! 난 홍길동이야."},
        {"role": "assistant", "content": "안녕하세요, 홍길동님! 반갑습니다."}
    ]
    
    # 세션에 대화 히스토리 추가
    await session.add_items(new_items)
    print("\n✓ 가상의 대화 히스토리가 세션에 추가됨")
    print(f"   - 사용자: {new_items[0]['content']}")
    print(f"   - 어시스턴트: {new_items[1]['content']}")
    
    # 이름을 묻는 질문 - 추가된 히스토리 기반으로 답변
    print("\n📝 질문: 내 이름이 뭐야?")
    result = await Runner.run(
        agent,
        "내 이름이 뭐야?",
        session=session
    )
    print(f"🤖 답변: {result.final_output}")
    
    print("\n" + "=" * 60)
    print("✓ 추가된 히스토리를 기반으로 이름을 기억함")
    print("=" * 60)

# 데모 실행
await demo_add_history()

대화 히스토리 추가 데모

✓ 가상의 대화 히스토리가 세션에 추가됨
   - 사용자: 안녕! 난 홍길동이야.
   - 어시스턴트: 안녕하세요, 홍길동님! 반갑습니다.

📝 질문: 내 이름이 뭐야?
🤖 답변: 홍길동님입니다!

✓ 추가된 히스토리를 기반으로 이름을 기억함


### 대화 히스토리 삭제하기

Session 객체의 `pop_item()` 메서드를 사용하여 마지막 대화 항목을 삭제할 수 있다.

In [6]:
async def demo_remove_history():
    """
    세션에서 대화 히스토리를 삭제하는 방법을 시연한다.
    """
    
    # 새로운 세션 생성
    session = SQLiteSession("demo_remove_history_001")
    
    print("=" * 60)
    print("대화 히스토리 삭제 데모")
    print("=" * 60)
    
    # 첫 번째 대화 - 이름 알려주기
    print("\n📝 첫 번째 대화: 안녕! 난 홍길동이야.")
    result1 = await Runner.run(
        agent,
        "안녕! 난 홍길동이야.",
        session=session
    )
    print(f"🤖 답변: {result1.final_output}")
    
    # 대화 히스토리 삭제
    # 비추론 모델: pop_item() 2번 (사용자 질문 + AI 응답)
    # 추론 모델: pop_item() 3번 (사용자 질문 + AI 추론 과정 + AI 응답)
    print("\n🗑️ 마지막 대화 히스토리 삭제 중...")
    
    try:
        assistant_item = await session.pop_item()  # AI 응답 제거
        user_item = await session.pop_item()       # 사용자 질문 제거
        print("   ✓ AI 응답 제거됨")
        print("   ✓ 사용자 질문 제거됨")
    except Exception as e:
        print(f"   삭제 중 에러: {e}")
    
    # 삭제 후 이름 묻기
    print("\n📝 삭제 후 질문: 내 이름은 뭐야?")
    result2 = await Runner.run(
        agent,
        "내 이름은 뭐야?",
        session=session
    )
    print(f"🤖 답변: {result2.final_output}")
    
    print("\n" + "=" * 60)
    print("✓ 히스토리 삭제 후 이름을 기억하지 못함")
    print("=" * 60)

# 데모 실행
await demo_remove_history()

대화 히스토리 삭제 데모

📝 첫 번째 대화: 안녕! 난 홍길동이야.
🤖 답변: 안녕, 홍길동! 반가워! 어떻게 지내?

🗑️ 마지막 대화 히스토리 삭제 중...
   ✓ AI 응답 제거됨
   ✓ 사용자 질문 제거됨

📝 삭제 후 질문: 내 이름은 뭐야?
🤖 답변: 죄송하지만, 당신의 이름을 알 수 없습니다.

✓ 히스토리 삭제 후 이름을 기억하지 못함


### 다중 세션 관리

서로 다른 세션 ID를 사용하여 여러 대화를 독립적으로 관리할 수 있다.

In [7]:
async def demo_multiple_sessions():
    """
    여러 세션을 독립적으로 관리하는 방법을 시연한다.
    """
    
    # 두 개의 독립된 세션 생성
    session_user_a = SQLiteSession("user_a_session")
    session_user_b = SQLiteSession("user_b_session")
    
    print("=" * 60)
    print("다중 세션 관리 데모")
    print("=" * 60)
    
    # 사용자 A 대화
    print("\n👤 사용자 A 세션")
    print("-" * 40)
    
    await Runner.run(
        agent,
        "안녕! 난 김철수야. 서울에 살아.",
        session=session_user_a
    )
    print("   사용자 A: 안녕! 난 김철수야. 서울에 살아.")
    
    # 사용자 B 대화
    print("\n👤 사용자 B 세션")
    print("-" * 40)
    
    await Runner.run(
        agent,
        "안녕! 난 이영희야. 부산에 살아.",
        session=session_user_b
    )
    print("   사용자 B: 안녕! 난 이영희야. 부산에 살아.")
    
    # 각 세션에서 정보 확인
    print("\n📝 각 세션에서 정보 확인")
    print("-" * 40)
    
    result_a = await Runner.run(
        agent,
        "내 이름과 사는 곳이 어디야?",
        session=session_user_a
    )
    print(f"   사용자 A 세션 답변: {result_a.final_output}")
    
    result_b = await Runner.run(
        agent,
        "내 이름과 사는 곳이 어디야?",
        session=session_user_b
    )
    print(f"   사용자 B 세션 답변: {result_b.final_output}")
    
    print("\n" + "=" * 60)
    print("✓ 각 세션이 독립적으로 대화 히스토리를 유지함")
    print("=" * 60)

# 데모 실행
await demo_multiple_sessions()

다중 세션 관리 데모

👤 사용자 A 세션
----------------------------------------
   사용자 A: 안녕! 난 김철수야. 서울에 살아.

👤 사용자 B 세션
----------------------------------------
   사용자 B: 안녕! 난 이영희야. 부산에 살아.

📝 각 세션에서 정보 확인
----------------------------------------
   사용자 A 세션 답변: 너의 이름은 김철수이고, 서울에 살고 있어.
   사용자 B 세션 답변: 이영희님은 부산에 살고 계십니다.

✓ 각 세션이 독립적으로 대화 히스토리를 유지함


---

## 2. 세션 저장하기

이 챕터에서는 세션을 외부에 저장하여 프로그램이 종료된 후에도 대화 이력을 복원하는 방법을 학습한다.

### 학습 내용

- 메모리 세션의 한계
- SQLite 데이터베이스에 세션 저장
- OpenAI Conversations API 사용
- 영구적인 대화 히스토리 관리

### 메모리 세션의 한계

기본 SQLiteSession은 메모리에서만 동작하므로 프로그램이 종료되면 세션 정보가 사라진다.

문제 상황:
  1. 프로그램 A: 첨성대가 경주에 있다고 대화
  2. 프로그램 A 종료
  3. 프로그램 B: '그 도시의 관광 명소는?' 질문
  4. 결과: 어떤 도시인지 알 수 없다고 답변

원인:
  - SQLiteSession("conversation_123")는 메모리에만 저장
  - 프로그램 종료 시 메모리 내 세션이 소멸
  - 동일한 세션 ID를 사용해도 맥락이 이어지지 않음

### SQLite 파일에 세션 저장하기

`db_path` 파라미터를 지정하여 세션을 SQLite 파일에 영구 저장할 수 있다.

In [9]:
async def demo_sqlite_persistent_session():
    """
    SQLite 파일에 세션을 저장하여 영구적으로 대화를 유지하는 방법을 시연한다.
    """
    
    # SQLite 파일 경로를 지정하여 세션 생성
    session = SQLiteSession(
        session_id="persistent_conversation_001",
        db_path="conversations.db"  # DB 파일 경로 지정
    )
    
    print("=" * 60)
    print("SQLite 파일 저장 세션 데모")
    print("=" * 60)
    print(f"\n📁 DB 파일: conversations.db")
    print(f"🔑 세션 ID: persistent_conversation_001")
    
    # 첫 번째 대화
    print("\n📝 질문: 첨성대는 어느 도시에 있나요?")
    result1 = await Runner.run(
        agent,
        "첨성대는 어느 도시에 있나요?",
        session=session
    )
    print(f"🤖 답변: {result1.final_output}")
    
    print("\n" + "=" * 60)
    print("✓ 대화가 conversations.db 파일에 저장됨")
    print("✓ 프로그램 재시작 후에도 세션 복원 가능")
    print("=" * 60)

# 데모 실행
await demo_sqlite_persistent_session()

SQLite 파일 저장 세션 데모

📁 DB 파일: conversations.db
🔑 세션 ID: persistent_conversation_001

📝 질문: 첨성대는 어느 도시에 있나요?
🤖 답변: 첨성대는 경주에 있습니다.

✓ 대화가 conversations.db 파일에 저장됨
✓ 프로그램 재시작 후에도 세션 복원 가능


### 저장된 세션 복원하기

In [10]:
async def demo_restore_session():
    """
    저장된 세션을 복원하여 이전 대화를 이어가는 방법을 시연한다.
    """
    
    # 동일한 세션 ID와 DB 파일로 세션 복원
    restored_session = SQLiteSession(
        session_id="persistent_conversation_001",
        db_path="conversations.db"
    )
    
    print("=" * 60)
    print("저장된 세션 복원 데모")
    print("=" * 60)
    print(f"\n📁 복원된 세션: persistent_conversation_001")
    
    # 이전 대화를 기반으로 질문
    print("\n📝 질문: 그 도시의 관광 명소를 알려주세요.")
    result = await Runner.run(
        agent,
        "그 도시의 관광 명소를 알려주세요.",
        session=restored_session
    )
    print(f"🤖 답변: {result.final_output}")
    
    print("\n" + "=" * 60)
    print("✓ 이전 대화('경주')를 기억하여 올바르게 답변함")
    print("=" * 60)

# 데모 실행
await demo_restore_session()

저장된 세션 복원 데모

📁 복원된 세션: persistent_conversation_001

📝 질문: 그 도시의 관광 명소를 알려주세요.
🤖 답변: 경주의 주요 관광 명소는 다음과 같습니다:

1. 불국사
2. 석굴암
3. 경주 동궁과 월지
4. 천마총
5. 경주 역사유적지구

이 외에도 많은 문화유산이 있습니다.

✓ 이전 대화('경주')를 기억하여 올바르게 답변함


### SQLite 데이터베이스 구조

저장된 대화 데이터베이스의 구조를 확인한다.

In [11]:
import sqlite3
import os

def inspect_database():
    """
    SQLite 데이터베이스의 내용을 확인한다.
    """
    
    db_path = "conversations.db"
    
    if not os.path.exists(db_path):
        print("데이터베이스 파일이 존재하지 않습니다.")
        return
    
    print("=" * 60)
    print("SQLite 데이터베이스 구조")
    print("=" * 60)
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # 테이블 목록 확인
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"\n📋 테이블 목록: {[t[0] for t in tables]}")
    
    # 세션 데이터 확인 (테이블이 존재하는 경우)
    for table in tables:
        table_name = table[0]
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        print(f"\n📊 테이블 '{table_name}': {count}개 레코드")
        
        # 샘플 데이터 조회
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 3")
        rows = cursor.fetchall()
        
        # 컬럼명 조회
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = [col[1] for col in cursor.fetchall()]
        print(f"   컬럼: {columns}")
        
        if rows:
            print("   샘플 데이터:")
            for row in rows:
                print(f"      {row[:2]}...")  # 처음 2개 필드만 표시
    
    conn.close()
    print("\n" + "=" * 60)

# 데이터베이스 확인
inspect_database()

SQLite 데이터베이스 구조

📋 테이블 목록: ['agent_sessions', 'agent_messages', 'sqlite_sequence']

📊 테이블 'agent_sessions': 1개 레코드
   컬럼: ['session_id', 'created_at', 'updated_at']
   샘플 데이터:
      ('persistent_conversation_001', '2025-12-14 11:41:21')...

📊 테이블 'agent_messages': 4개 레코드
   컬럼: ['id', 'session_id', 'message_data', 'created_at']
   샘플 데이터:
      (1, 'persistent_conversation_001')...
      (2, 'persistent_conversation_001')...
      (3, 'persistent_conversation_001')...

📊 테이블 'sqlite_sequence': 1개 레코드
   컬럼: ['name', 'seq']
   샘플 데이터:
      ('agent_messages', 4)...



### OpenAI Conversations API 사용하기

OpenAI의 Conversations API를 사용하여 서버 측에서 세션을 관리할 수 있다.

In [12]:
from openai import OpenAI

def create_conversation():
    """
    OpenAI Conversations API를 사용하여 Conversation 객체를 생성한다.
    """
    
    client = OpenAI()
    
    print("=" * 60)
    print("OpenAI Conversations API 사용")
    print("=" * 60)
    
    # Conversation 객체 생성
    try:
        conv = client.conversations.create()
        print(f"\n✓ Conversation 생성 완료")
        print(f"🔑 Conversation ID: {conv.id}")
        return conv.id
    except Exception as e:
        print(f"\n✗ Conversation 생성 실패: {e}")
        print("   (Conversations API가 지원되지 않는 계정일 수 있음)")
        return None

# Conversation 생성
conversation_id = create_conversation()

OpenAI Conversations API 사용

✓ Conversation 생성 완료
🔑 Conversation ID: conv_693ea28aaf208195832385a319de78ea00eab1bf66b3c12b


### OpenAI Conversations Session 사용하기

In [13]:
# OpenAIConversationsSession이 사용 가능한 경우에만 실행
try:
    from agents import OpenAIConversationsSession
    CONVERSATIONS_AVAILABLE = True
except ImportError:
    CONVERSATIONS_AVAILABLE = False
    print("OpenAIConversationsSession을 사용할 수 없습니다.")

async def demo_conversations_api():
    """
    OpenAI Conversations API를 사용한 세션 관리를 시연한다.
    """
    
    if not CONVERSATIONS_AVAILABLE:
        print("OpenAIConversationsSession이 지원되지 않습니다.")
        return
    
    if not conversation_id:
        print("Conversation ID가 없습니다. 먼저 Conversation을 생성하세요.")
        return
    
    # Conversation 기반 에이전트
    conversations_agent = Agent(
        name="Conversations Assistant",
        instructions="매우 간결하게 답변해 주세요. 한국어로 응답하세요.",
        model="gpt-4o-mini"  # gpt-5 모델 사용 시 오류 발생 가능
    )
    
    # OpenAI Conversations Session 생성
    session = OpenAIConversationsSession(conversation_id=conversation_id)
    
    print("=" * 60)
    print("OpenAI Conversations Session 데모")
    print("=" * 60)
    print(f"\n🔑 Conversation ID: {conversation_id}")
    
    # 첫 번째 대화
    print("\n📝 질문: 첨성대는 어느 도시에 있나요?")
    result1 = await Runner.run(
        conversations_agent,
        "첨성대는 어느 도시에 있나요?",
        session=session
    )
    print(f"🤖 답변: {result1.final_output}")
    
    # 두 번째 대화
    print("\n📝 질문: 그 도시의 관광 명소를 알려주세요.")
    result2 = await Runner.run(
        conversations_agent,
        "그 도시의 관광 명소를 알려주세요.",
        session=session
    )
    print(f"🤖 답변: {result2.final_output}")
    
    print("\n" + "=" * 60)
    print("✓ OpenAI 서버에서 대화 히스토리가 관리됨")
    print("=" * 60)

# 데모 실행 (Conversations API가 사용 가능한 경우)
if CONVERSATIONS_AVAILABLE and conversation_id:
    await demo_conversations_api()
else:
    print("Conversations API 데모를 건너뜁니다.")

OpenAI Conversations Session 데모

🔑 Conversation ID: conv_693ea28aaf208195832385a319de78ea00eab1bf66b3c12b

📝 질문: 첨성대는 어느 도시에 있나요?
🤖 답변: 첨성대는 경주에 있습니다.

📝 질문: 그 도시의 관광 명소를 알려주세요.
🤖 답변: 경주의 주요 관광 명소는 다음과 같습니다:

1. 경주 불국사
2. 석굴암
3. 안압지
4. 대릉원
5. 경주 국립박물관
6. 첨성대
7. 포석정

각각의 명소에서 역사와 문화유산을 체험할 수 있습니다.

✓ OpenAI 서버에서 대화 히스토리가 관리됨


### conversation_id 직접 사용하기

일부 모델에서는 session 대신 conversation_id를 직접 지정해야 할 수 있다.

In [14]:
async def demo_direct_conversation_id():
    """
    conversation_id를 직접 지정하여 대화하는 방법을 시연한다.
    GPT-5 모델 사용 시 session 대신 이 방법을 사용해야 할 수 있다.
    """
    
    if not conversation_id:
        print("Conversation ID가 없습니다.")
        return
    
    print("=" * 60)
    print("conversation_id 직접 사용 데모")
    print("=" * 60)
    print(f"\n🔑 Conversation ID: {conversation_id}")
    
    # session 대신 conversation_id 직접 지정
    print("\n📝 질문: 안녕하세요!")
    try:
        result = await Runner.run(
            agent,
            "안녕하세요!",
            conversation_id=conversation_id  # session 대신 직접 지정
        )
        print(f"🤖 답변: {result.final_output}")
        print("\n✓ conversation_id 직접 지정 방식 성공")
    except Exception as e:
        print(f"\n✗ 에러: {e}")
    
    print("\n" + "=" * 60)
    print("참고: GPT-5 모델 사용 시 session 대신 conversation_id 사용 권장")
    print("=" * 60)

# 데모 실행
if conversation_id:
    await demo_direct_conversation_id()

conversation_id 직접 사용 데모

🔑 Conversation ID: conv_693ea28aaf208195832385a319de78ea00eab1bf66b3c12b

📝 질문: 안녕하세요!
🤖 답변: 안녕하세요! 어떻게 도와드릴까요?

✓ conversation_id 직접 지정 방식 성공

참고: GPT-5 모델 사용 시 session 대신 conversation_id 사용 권장


---

## 세션 저장 방식 비교

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 세션 관리를 다루었다. 다음과 같은 내용을 학습하였다:

1. **세션 사용하기**
   - SQLiteSession을 사용한 대화 히스토리 관리
   - 세션 ID를 통한 대화 묶음 구분
   - add_items(), pop_item()으로 히스토리 관리
   - 다중 세션 독립 관리

2. **세션 저장하기**
   - 메모리 세션의 한계
   - db_path 파라미터로 SQLite 파일 저장
   - OpenAI Conversations API 사용
   - conversation_id 직접 지정 방식

세션을 활용하면 대화형 에이전트에서 자연스러운 맥락 유지가 가능하며, 적절한 저장 방식을 선택하여 요구사항에 맞는 영구 저장을 구현할 수 있다.